In [1]:
# ColabをDrive内で使う
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

# 作業フォルダを作る（初回のみ）
# !mkdir -p cat-variety
# %cd cat-variety


Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
# # Kaggleからデータセットをダウンロード
# # Google Colabの基本確認
# import sys, platform
# print("Python version:", sys.version)
# print("Platform:", platform.platform())

# # GPU確認
# !nvidia-smi

# # Kaggle APIのインストール
# !pip install kaggle

# from google.colab import files
# files.upload()  # kaggle.jsonをアップロード

# # 認証設定
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# # データセットをColabにダウンロード
# !kaggle datasets download -d ma7555/cat-breeds-dataset --unzip


Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.6.97+-x86_64-with-glibc2.35
Mon Oct 13 09:36:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                          

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/ma7555/cat-breeds-dataset
License(s): DbCL-1.0
 99% 1.92G/1.93G [00:12<00:00, 200MB/s]
100% 1.93G/1.93G [00:12<00:00, 168MB/s]


In [5]:
# 必要な画像のみをDriveに保存
import os
import shutil

# 元データの場所（Kaggleから展開したフォルダ）
src_dir = "/content/drive/MyDrive/cat-variety/images/images"

# 保存先（Google Drive）
dst_dir = "/content/drive/MyDrive/cat-variety/data/train"
os.makedirs(dst_dir, exist_ok=True)

# 抽出する7品種
selected_breeds = [
    "Scottish Fold",
    "American Shorthair",
    "Russian Blue",
    "Siamese",
    "Persian",
    "Maine Coon",
    "Norwegian Forest Cat"
]

# コピー処理
for breed in selected_breeds:
    src_path = os.path.join(src_dir, breed)
    dst_path = os.path.join(dst_dir, breed)
    if os.path.exists(src_path):
        shutil.copytree(src_path, dst_path)
        print(f"✅ {breed} をコピーしました。")
    else:
        print(f"⚠️ {breed} が見つかりません。")

print("\n整理完了。Drive内フォルダ:")
print(os.listdir(dst_dir))


✅ Scottish Fold をコピーしました。
✅ Russian Blue をコピーしました。
✅ Siamese をコピーしました。
✅ Persian をコピーしました。
✅ Maine Coon をコピーしました。
✅ Norwegian Forest Cat をコピーしました。

整理完了。Drive内フォルダ:
['American Shorthair', 'Scottish Fold', 'Russian Blue', 'Siamese', 'Persian', 'Maine Coon', 'Norwegian Forest Cat']


In [6]:
# 基本ライブラリ
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# GPU確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 画像前処理
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 画像サイズを統一
    transforms.ToTensor(),           # Tensor化
    transforms.Normalize(            # 標準化
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# データセット読み込み
train_dir = "/content/drive/MyDrive/cat-variety/data/train"
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# ローダー作成
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"画像枚数: {len(train_dataset)}")
print(f"クラス一覧: {train_dataset.classes}")


Using device: cuda
画像枚数: 16449
クラス一覧: ['American Shorthair', 'Maine Coon', 'Norwegian Forest Cat', 'Persian', 'Russian Blue', 'Scottish Fold', 'Siamese']


In [8]:
# モデル定義

import torch.nn as nn
from torchvision import models

# 事前学習済みResNet18をロード
model = models.resnet18(pretrained=True)

# 出力層を品種数に合わせて置き換え
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# GPU対応
model = model.to(device)

print(model)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# 損失関数と最適化アルゴリズム」の設定
import torch.optim as optim

# 損失関数（分類問題なのでクロスエントロピー）
criterion = nn.CrossEntropyLoss()

# 最適化手法（SGDより安定するAdamを使用/Adam：各パラメータごとに最適な学習率を自動調整）
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("損失関数:", criterion)
print("最適化手法:", optimizer)


損失関数: CrossEntropyLoss()
最適化手法: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


In [10]:
# 学習ループ
num_epochs = 1  # 初回は1で確認
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()           # 勾配リセット
        outputs = model(images)         # 推論
        loss = criterion(outputs, labels)  # 損失計算
        loss.backward()                 # 逆伝播
        optimizer.step()                # 重み更新

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/1], Loss: 0.6534


In [11]:
# 学習データを訓練用と検証用に分割
from torch.utils.data import random_split

# 全データを8:2に分割
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# 検証用データローダー
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

print(f"train: {len(train_subset)}, val: {len(val_subset)}")


train: 13159, val: 3290


In [12]:
# 検証データでの精度計算
model.eval()  # 評価モード
correct = 0
total = 0

with torch.no_grad():  # 勾配計算オフ
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"検証データ精度: {accuracy:.2f}%")


検証データ精度: 90.70%


In [13]:
# エポックを増やして精度推移を見る
num_epochs = 5  # 試しに5回
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 検証
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss/len(train_loader):.4f}  Val Acc: {acc:.2f}%")


Epoch [1/5] Loss: 0.3074  Val Acc: 96.60%
Epoch [2/5] Loss: 0.1256  Val Acc: 99.03%
Epoch [3/5] Loss: 0.0641  Val Acc: 97.42%
Epoch [4/5] Loss: 0.0545  Val Acc: 99.18%
Epoch [5/5] Loss: 0.0623  Val Acc: 98.91%


In [15]:
# 学習済みモデルの保存
# Driveへの保存パス
save_path = "/content/drive/MyDrive/cat-variety/model/cat_model.pth"

# モデル保存
torch.save(model.state_dict(), save_path)
print(f"モデルを保存しました: {save_path}")


モデルを保存しました: /content/drive/MyDrive/cat-variety/model/cat_model.pth
